In [14]:
# 필요한 라이브러리와 모듈을 임포트합니다.
import os
import numpy as np
import pandas as pd
import ast
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras_tuner import RandomSearch, HyperModel
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU

In [3]:
# 주어진 디렉토리에서 데이터를 로드하고 전처리하는 함수입니다.
def load_and_preprocess_data(directory, sample_fraction=0.01):
    all_drawings = []  # 모든 그림 데이터를 저장할 리스트
    all_labels = []  # 모든 라벨(단어)을 저장할 리스트
    # 디렉토리 내의 모든 파일을 순회합니다.
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):  # CSV 파일인 경우
            filepath = os.path.join(directory, filename)  # 파일 경로 생성
            # CSV 파일을 읽어 들이면서 'drawing'과 'word' 컬럼만 사용합니다.
            data = pd.read_csv(filepath, usecols=['drawing', 'word'], nrows=100)  # 첫 100개의 row만 로드
            data['drawing'] = data['drawing'].apply(ast.literal_eval)  # 'drawing' 컬럼의 문자열을 파이썬 객체로 변환
            all_labels.extend(data['word'].tolist())  # 라벨(단어) 데이터를 리스트에 추가
            for drawing in data['drawing']:  # 각 그림 데이터에 대해
                image = drawing_to_np(drawing, size=64)  # 그림 데이터를 넘파이 배열 이미지로 변환
                all_drawings.append(image)  # 변환된 이미지를 리스트에 추가
    return np.array(all_drawings), np.array(all_labels)  # 넘파이 배열로 변환하여 반환

In [4]:
# 'drawing' 데이터를 넘파이 배열로 변환하는 함수입니다.
def drawing_to_np(drawing, size=64):
    img = np.zeros((256, 256), dtype=np.uint8)  # 초기 이미지 배열 생성
    for stroke in drawing:  # 각 스트로크에 대해
        for x, y in zip(stroke[0], stroke[1]):  # 스트로크의 x, y 좌표에 대해
            img[y, x] = 255  # 해당 좌표를 흰색으로 채움
    img = tf.image.resize(img[..., np.newaxis], (size, size)).numpy()  # 이미지 크기를 조정
    img = img / 255.0  # 이미지를 정규화
    return img.squeeze()  # 불필요한 차원 제거 후 반환

In [5]:
class MyHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        # 모델의 입력 형태를 초기화합니다.
        self.input_shape = input_shape
        # 모델이 분류해야 할 클래스의 개수를 초기화합니다.
        self.num_classes = num_classes
        
    def build(self, hp):
        # Sequential 모델을 생성합니다.
        model = Sequential([
            # 첫 번째 Conv2D 층: 하이퍼파라미터 탐색을 통해 필터의 수를 결정합니다.
            Conv2D(filters=hp.Int('conv_1_filters', min_value=32, max_value=128, step=32),
                   kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),  # 커널 크기를 하이퍼파라미터 탐색으로 결정
                   padding='same',  # 입력과 출력의 공간적 차원을 동일하게 유지
                   input_shape=self.input_shape),  # 입력 형태를 모델에 전달
            LeakyReLU(alpha=0.01),  # LeakyReLU 활성화 함수 적용, 작은 음수 기울기를 허용
            MaxPooling2D(pool_size=(2, 2)),  # 공간적 차원 축소를 위한 MaxPooling 적용
            Dropout(hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)),  # 과적합 방지를 위한 Dropout 적용
            
            # 두 번째 Conv2D 층
            Conv2D(filters=hp.Int('conv_2_filters', min_value=64, max_value=256, step=32),
                   kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
                   padding='same'),
            LeakyReLU(alpha=0.01),  # 두 번째 Conv2D 층에도 LeakyReLU 활성화 함수 적용
            MaxPooling2D(pool_size=(2, 2)),
            Dropout(hp.Float('dropout_2', min_value=0.0, max_value=0.5, step=0.1)),
            
            # 특성 맵을 1D 벡터로 평탄화
            Flatten(),
            # 완전 연결(Dense) 층: 유닛 수는 하이퍼파라미터 탐색을 통해 결정
            Dense(units=hp.Int('dense_units', min_value=100, max_value=512, step=64)),
            LeakyReLU(alpha=0.01),  # Dense 층 후 LeakyReLU 적용
            Dropout(hp.Float('dropout_3', min_value=0.0, max_value=0.5, step=0.1)),  # 마지막 Dropout 적용
            # 출력 층: 클래스 수에 맞는 출력 유닛과 softmax 활성화 함수 사용
            Dense(self.num_classes, activation='softmax'),
        ])
        
        # 모델 컴파일: Adam 옵티마이저와 범주형 교차 엔트로피 손실 함수 사용
        model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model  # 구축된 모델을 반환

In [6]:
# 데이터 로드 및 전처리
images, labels = load_and_preprocess_data('C:\\Users\\OWJ\\Downloads\\data\\train_simplified')
label_encoder = LabelEncoder()  # 라벨 인코더 인스턴스 생성
encoded_labels = label_encoder.fit_transform(labels)  # 라벨을 정수 인코딩
categorical_labels = to_categorical(encoded_labels)  # 정수 인코딩된 라벨을 원-핫 인코딩
X_train, X_val, y_train, y_val = train_test_split(images, categorical_labels, test_size=0.2, random_state=42)  # 데이터를 훈련 세트와 검증 세트로 분할

In [7]:
# X_train과 X_val 배열을 4차원으로 변경
X_train = X_train.reshape(-1, 64, 64, 1)  # -1은 샘플 수를 자동으로 계산하도록 합니다.
X_val = X_val.reshape(-1, 64, 64, 1)

# 데이터 증강 설정
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# 데이터 증강 적용
train_generator = datagen.flow(X_train, y_train, batch_size=32)
# 검증 데이터셋에는 rescale만 적용 (증강 적용 X)
val_gen = ImageDataGenerator(rescale=1./255).flow(X_val, y_val, batch_size=32)

In [8]:
# 학습률을 동적으로 조절하는 스케줄러 함수
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [9]:
# 하이퍼파라미터 튜닝 설정
hypermodel = MyHyperModel(input_shape=(64, 64, 1), num_classes=len(np.unique(labels)))  # 하이퍼모델 인스턴스 생성
tuner = RandomSearch(
    hypermodel,  # 하이퍼모델 지정
    objective='val_accuracy',  # 목표는 검증 세트의 정확도 최대화
    max_trials=10,  # 시도할 최대 트라이얼 수
    executions_per_trial=2,  # 트라이얼 당 실행 횟수
    directory='my_dir',  # 튜닝 로그를 저장할 디렉토리
    project_name='quick_draw_tuning'  # 프로젝트 이름
)

Reloading Tuner from my_dir\quick_draw_tuning\tuner0.json


In [10]:
# 콜백 설정
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    LearningRateScheduler(lr_scheduler)
]

In [12]:
tuner.search(x=train_generator, epochs=10, validation_data=(X_val, y_val), callbacks=[callbacks])  # 하이퍼파라미터 검색 시작

In [15]:
# 최적의 하이퍼파라미터로 모델 훈련
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]  # 최적의 하이퍼파라미터 추출
model = tuner.hypermodel.build(best_hps)  # 최적의 하이퍼파라미터로 모델 구축
model.fit(
    train_generator,
    epochs=50,
    validation_data=val_gen,
    callbacks=callbacks
)  # 모델 훈련


Epoch 1/50


850/850 [==============================] - 22s 25ms/step - loss: 5.7368 - accuracy: 0.0072 - val_loss: 6.6435 - val_accuracy: 0.0016 - lr: 0.0020
Epoch 2/50
850/850 [==============================] - 37s 43ms/step - loss: 5.5469 - accuracy: 0.0134 - val_loss: 6.9852 - val_accuracy: 0.0016 - lr: 0.0020
Epoch 3/50
850/850 [==============================] - 53s 63ms/step - loss: 5.4846 - accuracy: 0.0160 - val_loss: 7.3308 - val_accuracy: 0.0016 - lr: 0.0020
Epoch 4/50
850/850 [==============================] - 57s 67ms/step - loss: 5.4412 - accuracy: 0.0186 - val_loss: 7.1473 - val_accuracy: 0.0016 - lr: 0.0020
Epoch 5/50
850/850 [==============================] - 62s 73ms/step - loss: 5.3957 - accuracy: 0.0214 - val_loss: 7.3047 - val_accuracy: 0.0016 - lr: 0.0020
Epoch 6/50
850/850 [==============================] - 65s 76ms/step - loss: 5.3748 - accuracy: 0.0240 - val_loss: 7.5755 - val_accuracy: 0.0016 - lr: 0.0020


In [16]:
# 예측된 클래스 확률을 기반으로 MAP@3 점수를 계산하는 함수
def calculate_map3_for_batch(predictions, true_labels):
    total_score = 0.0
    for i in range(predictions.shape[0]):  # 배치 내 각 샘플에 대해
        top_3_preds_indices = np.argsort(predictions[i])[-3:][::-1]  # 상위 3개 예측
        true_label = np.argmax(true_labels[i])  # 실제 라벨
        score = 0.0
        if true_label in top_3_preds_indices:  # 실제 라벨이 상위 3개 예측에 포함되는 경우
            rank = np.where(top_3_preds_indices == true_label)[0][0] + 1  # 랭크 계산
            score = 1.0 / rank  # 점수 계산
        total_score += score
    map3 = total_score / predictions.shape[0]  # 평균 점수 계산
    return map3

In [17]:
# 검증 데이터 제너레이터를 사용하여 모델의 MAP@3 점수를 계산하는 함수
def evaluate_model_with_map3(model, validation_generator, X_val):
    total_map3_score = 0.0
    num_batches = 0
    validation_steps = len(X_val) // validation_generator.batch_size  # 처리할 전체 배치 수 계산
    
    for batch_images, batch_labels in validation_generator:  # 검증 데이터 제너레이터에서 배치를 반복해서 가져옴
        predictions = model.predict(batch_images)  # 현재 배치에 대한 예측
        map3_score = calculate_map3_for_batch(predictions, batch_labels)  # 현재 배치에 대한 MAP@3 점수 계산
        total_map3_score += map3_score  # 총 점수에 추가
        num_batches += 1
        
        if num_batches >= validation_steps:  # 모든 배치를 처리했으면 종료
            break
    
    average_map3_score = total_map3_score / num_batches  # 평균 MAP@3 점수 계산
    return average_map3_score

In [18]:
# 모델과 검증 데이터셋의 제너레이터를 함수에 전달하여 평가
average_map3_score = evaluate_model_with_map3(model, val_gen, X_val)
print(f"Average MAP@3 Score across all batches: {average_map3_score}")  # 평균 MAP@3 점수 출력

1/1 [==============================] - 0s 20ms/step
Average MAP@3 Score across all batches: 0.0038571147798742134
